# 환경설정

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

# 크롬창 열기

In [26]:
# css 찾을때 까지 10초대기
def time_wait(num, code,driver):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [27]:
def find_search_input(key_word,driver):
    # 검색창 찾기
    driver.switch_to.default_content()
    time_wait(2, 'div.input_box > input.input_search',driver)
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    driver.implicitly_wait(3)
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    sleep(5)
    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
    sleep(1)

In [28]:
# frame 변경 메소드
def switch_frame(frame,driver):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    driver.implicitly_wait(1)


In [29]:
# 페이지 다운
def page_down(num,driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [35]:
# 페이지 다운
def page_end(driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    body.send_keys(Keys.END)

In [36]:
def find_review_btn(driver):
    sleep(2)
    time_wait(10,'a.tpj9w',driver)
    menu_list = driver.find_elements(By.CSS_SELECTOR,'a.tpj9w')
    if len(menu_list) ==7:
        return menu_list[-1]
    else:
        return menu_list[-2]

In [37]:
def crawling_visitor_review(driver):
    #-----리뷰 버튼 클릭-------
    find_review_btn(driver).click()
    SCROLL_PAUSE_SEC = 1
    
    last_scroll = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        
        try :
            #더보기 버튼 클릭
            driver.find_element(By.CSS_SELECTOR,'a.fvwqf').click()
            
        except:
            #더보기 버튼이 없다면 while문을 빠져나온다.
            break
            # 끝까지 스크롤 다운
            
        new_scroll = driver.execute_script("return document.body.scrollHeight")
        if last_scroll == new_scroll:
            break
        else:
            last_scroll= new_scroll



    #가게 리스트 마다 방문자 리뷰 가져오기
    review_elements = driver.find_elements(By.CSS_SELECTOR,"div.ZZ4OK > a > span")
    review_list = []
    for i in review_elements:
        review_list.append(i.text)
    
    return review_list


# 블로그 리뷰 크롤링 함수
## 불필요한 코퍼스가 많다.

In [10]:
'''
def crawling_blog_review():
    SCROLL_PAUSE_SEC = 3
    sleep(SCROLL_PAUSE_SEC)
    try :
        blog_review_button = driver.find_element(By.CSS_SELECTOR, 'div._1tclc > div > a:nth-child(2)')
        blog_review_button.click()
    except:
        print('블로그 리뷰가 없습니다.')
        return
    else:
        driver.implicitly_wait(SCROLL_PAUSE_SEC)

        last_scroll = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

            try :
                #더보기 버튼 클릭
                driver.find_element(By.CSS_SELECTOR,'a._3iTUo').click()
                driver.implicitly_wait(SCROLL_PAUSE_SEC)
            except:
                break
                # 끝까지 스크롤 다운

            new_scroll = driver.execute_script("return document.body.scrollHeight")
            driver.implicitly_wait(SCROLL_PAUSE_SEC)
            if last_scroll == new_scroll:
                break
            else:
                last_scroll = new_scroll

        #가게 리스트 마다 방문자 리뷰 가져오기
        review_elements = driver.find_elements(By.CSS_SELECTOR,'div._3jvFk')
        review_list = []
        for i in review_elements:
            review_list.append(i.text)
        return review_list
'''

'\ndef crawling_blog_review():\n    SCROLL_PAUSE_SEC = 3\n    sleep(SCROLL_PAUSE_SEC)\n    try :\n        blog_review_button = driver.find_element(By.CSS_SELECTOR, \'div._1tclc > div > a:nth-child(2)\')\n        blog_review_button.click()\n    except:\n        print(\'블로그 리뷰가 없습니다.\')\n        return\n    else:\n        driver.implicitly_wait(SCROLL_PAUSE_SEC)\n\n        last_scroll = driver.execute_script("return document.body.scrollHeight")\n        while True:\n            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")\n            time.sleep(1)\n\n            try :\n                #더보기 버튼 클릭\n                driver.find_element(By.CSS_SELECTOR,\'a._3iTUo\').click()\n                driver.implicitly_wait(SCROLL_PAUSE_SEC)\n            except:\n                break\n                # 끝까지 스크롤 다운\n\n            new_scroll = driver.execute_script("return document.body.scrollHeight")\n            driver.implicitly_wait(SCROLL_PAUSE_SEC)\n            if last_s

# 검색 지역 횟집의 리뷰를 크롤링.

In [143]:
'''
def review_crawling(key_word,page_num):
    WAIT=3
    #검색창을 찾아서 서치
    find_search_input(key_word)
    # searchIframe 변경
    driver.implicitly_wait(2)
    switch_frame('searchIframe')
    driver.implicitly_wait(WAIT)
    driver.execute_script("window.scrollTo(0, 0);")
    driver.implicitly_wait(WAIT)
    # 페이지 버튼의 수
    pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
    page_num = page_num-1
    if page_num <= 5:
        pages[page_num].click()
        driver.implicitly_wait(WAIT)
    else:
        driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
        driver.implicitly_wait(WAIT)
        driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
        driver.implicitly_wait(WAIT)
        
    driver.implicitly_wait(2)
    page_down(45)
    driver.implicitly_wait(WAIT)
    page_down(5)
    driver.implicitly_wait(2)
    store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
    print('first len of store_elements : ', len(store_elements))
    driver.implicitly_wait(2)
    #한 페이지의 50개의 매장들 하나씩 조회
    for idx in range(len(store_elements)):
        driver.implicitly_wait(1)
        store_elements[idx].click() #li의 스토어를 클릭
        driver.implicitly_wait(WAIT) # 2초 휴식
        try:
            #다른 iframe으로 변경
            switch_frame('entryIframe')
            time_wait(2, '.Fc1rA')

            # -----매장명 가져오기-----
            store_name = driver.find_element(By.CSS_SELECTOR,'.Fc1rA').text
            driver.implicitly_wait(2)
            review_dic[store_name] = {}

            # -----위치 가져오기-----
            try : 
                location = driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text
                driver.implicitly_wait(WAIT)
                review_dic[store_name]['location'] = location
            except:
                pass

            # -----리뷰 크롤링---------
            # 방문자 리뷰 크롤링
            review_dic[store_name]['review'] = crawling_visitor_review()
            driver.implicitly_wait(WAIT)
            #driver.execute_script("window.scrollTo(0, 0);")
            #driver.implicitly_wait(WAIT)
            # 블로거 리뷰 크롤링
            #review_dic[store_name]['review'] = review_dic[store_name]['review']+crawling_blog_review()
            #driver.implicitly_wait(WAIT)

        except:
            pass
        switch_frame('searchIframe')
        refresh_point = (len(store_elements)-1)//3

        if idx == refresh_point:
            sleep(3)
            driver.refresh()
            driver.implicitly_wait(3)
            switch_frame('searchIframe')
            driver.implicitly_wait(3)
            driver.refresh()
            driver.implicitly_wait(2)
            switch_frame('searchIframe')
            driver.implicitly_wait(2)
            body = driver.find_element(By.CSS_SELECTOR,'body.place_on_pcmap')
            body.click()
            for i in range(40): 
                body.send_keys(Keys.PAGE_DOWN)
            driver.implicitly_wait(3)
            #driver.implicitly_wait(1)
            for i in range(5): 
                body.send_keys(Keys.PAGE_DOWN)
            store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
            driver.implicitly_wait(3)
            print('len of store_elements : ', len(store_elements))
            driver.implicitly_wait(3)

        elif idx == (refresh_point*2):
            sleep(3)
            driver.refresh()
            driver.implicitly_wait(3)
            switch_frame('searchIframe')
            driver.implicitly_wait(3)
            driver.refresh()
            driver.implicitly_wait(2)
            switch_frame('searchIframe')
            driver.implicitly_wait(2)
            body = driver.find_element(By.CSS_SELECTOR,'body.place_on_pcmap')
            body.click()
            for i in range(40): 
                body.send_keys(Keys.PAGE_DOWN)
            driver.implicitly_wait(3)
            #driver.implicitly_wait(1)
            for i in range(5): 
                body.send_keys(Keys.PAGE_DOWN)
            store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
            driver.implicitly_wait(3)
            print('len of store_elements : ', len(store_elements))
            driver.implicitly_wait(3)
            
        # json 파일로 저장
    with open(f'naver_review_dic_page{page_nums}.json', 'w', encoding='utf-8') as f:
        json.dump(review_dic, f, indent=4, ensure_ascii=False)
        
'''            

In [140]:
# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
#options = webdriver.ChromeOptions()
#options.add_argument('headless')
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_14752\145050225.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [47]:
review_dic=dict()
store_elements = list()
def review_crawling2(key_word, page_num):
    global review_dic
    global store_elements
    WAIT = 3
    page_num = page_num-1
    for idx in range(3):
        url = 'https://map.naver.com/v5/search'
        driver = webdriver.Chrome('./chromedriver')
        driver.get(url)
        driver.implicitly_wait(2)
        
        find_search_input(key_word,driver=driver)
        driver.implicitly_wait(2)
        switch_frame('searchIframe',driver)
        driver.implicitly_wait(WAIT)
        driver.execute_script("window.scrollTo(0, 0);")
        driver.implicitly_wait(WAIT)
         # 페이지 버튼의 수
        pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
        if page_num <= 5:
            pages[page_num].click()
            driver.implicitly_wait(WAIT)
        else:
            driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
            driver.implicitly_wait(WAIT)
            driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
            driver.implicitly_wait(WAIT)
            
        driver.implicitly_wait(2)
        page_down(40,driver)
        driver.implicitly_wait(WAIT)
        page_down(5,driver)
        driver.implicitly_wait(2)
        store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
        print('total store_list : ', len(store_elements))
        
        
        
        ch_pt1 = len(store_elements) // 3
        ch_pt2 = ch_pt1*2
        if idx == 0:
            print('first crawling store_list', ch_pt1)
            for i in range(ch_pt1):
                driver.implicitly_wait(1)
                store_elements[i].click() #li의 스토어를 클릭
                driver.implicitly_wait(WAIT) # 2초 휴식
                try:
                    #다른 iframe으로 변경
                    switch_frame('entryIframe',driver)
                    time_wait(2, '.Fc1rA',driver)

                    # -----매장명 가져오기-----
                    store_name = driver.find_element(By.CSS_SELECTOR,'.Fc1rA').text
                    driver.implicitly_wait(2)
                    review_dic[store_name] = {}

                    # -----위치 가져오기-----
                    try : 
                        location = driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text
                        driver.implicitly_wait(WAIT)
                        review_dic[store_name]['location'] = location
                    except:
                        pass

                    # -----리뷰 크롤링---------
                    # 방문자 리뷰 크롤링
                    review_dic[store_name]['review'] = crawling_visitor_review(driver)
                    driver.implicitly_wait(WAIT)
                except:
                    pass
                switch_frame('searchIframe',driver)
                
                    
                    
                    
        elif idx == 1:
            print('second crawling store_list', ch_pt2-ch_pt1)
            for i in range(ch_pt1,ch_pt2):
                driver.implicitly_wait(1)
                store_elements[i].click() #li의 스토어를 클릭
                driver.implicitly_wait(WAIT) # 2초 휴식
                try:
                    #다른 iframe으로 변경
                    switch_frame('entryIframe',driver)
                    time_wait(2, '.Fc1rA',driver)

                    # -----매장명 가져오기-----
                    store_name = driver.find_element(By.CSS_SELECTOR,'.Fc1rA').text
                    driver.implicitly_wait(2)
                    review_dic[store_name] = {}

                    # -----위치 가져오기-----
                    try : 
                        location = driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text
                        driver.implicitly_wait(WAIT)
                        review_dic[store_name]['location'] = location
                    except:
                        pass

                    # -----리뷰 크롤링---------
                    # 방문자 리뷰 크롤링
                    review_dic[store_name]['review'] = crawling_visitor_review(driver)
                    driver.implicitly_wait(WAIT)
                except:
                    pass
                switch_frame('searchIframe',driver)
            
        else :
            print('third crawling store_list', len(store_elements)-ch_pt2)
            for i in range(ch_pt2,len(store_elements)):
                driver.implicitly_wait(1)
                store_elements[i].click() #li의 스토어를 클릭
                driver.implicitly_wait(WAIT) # 2초 휴식
                try:
                   #다른 iframe으로 변경
                    switch_frame('entryIframe',driver)
                    time_wait(2, '.Fc1rA',driver)

                    # -----매장명 가져오기-----
                    store_name = driver.find_element(By.CSS_SELECTOR,'.Fc1rA').text
                    driver.implicitly_wait(2)
                    review_dic[store_name] = dict()

                    # -----위치 가져오기-----
                    try : 
                        location = driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text
                        driver.implicitly_wait(WAIT)
                        review_dic[store_name]['location'] = location
                    except:
                        pass

                    # -----리뷰 크롤링---------
                    # 방문자 리뷰 크롤링
                    review_dic[store_name]['review'] = crawling_visitor_review(driver)
                    driver.implicitly_wait(WAIT)
                except:
                    pass
                switch_frame('searchIframe',driver)
                
                    
    # json 파일로 저장
    with open(f'naver_{key_word}_review_dic_page{page_num}.json', 'w', encoding='utf-8') as f:
        json.dump(review_dic, f, indent=4, ensure_ascii=False)
        
            
        
        
    
    
    

In [48]:
review_crawling2('서울 횟집',1)

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_4904\3982112359.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  54
first crawling store_list 18
total store_list :  54
second crawling store_list 18
total store_list :  54
third crawling store_list 18


In [141]:
key_word_list = ['서울 횟집', '인천 횟집', '부산 횟집', '대구 횟집', ' 대전 횟집', '울산 횟집', '광주 횟집']

In [49]:
#review_crawling(key_word_list[0],1)

In [51]:
len(review_dic)

51

In [53]:
review_dic.keys()

dict_keys(['목포회센타', '옛날마차신관', '해누리', '횟집울릉도', '바다회사랑', '은성횟집', '진동횟집', '최우영수산', '어사랑횟집', '건대사시미', '상훈이네 바다이야기', '자매수산', '횟집산이네', '기유', '어다리횟집구로점', '으뜸횟집', '옛날마차', '회뜨는사나이 1호점', '회통령 홍대점', '동해수산', '오 막회집 서울대점', '두꺼비숙성횟집 마곡점', '목포명가 세곡점', '강남보물섬', '오징어데이', '형제상회', '이수회관', '피쉬스토리', '다성 대게킹크랩', '고기야미안해', '수다떠는오징어', '바다회세꼬시', '바다풍경', '남대문막내횟집 광교점', '매일스시횟집', '바다회사랑2호점', '유명한횟집', '남해안', '제주식당', '진동둔횟집', '오징어선수촌 방이점', '광어포차', '부부횟집', '응수수산', '오늘은내가쏜다', '회뜰때까지', '가산어시장', '선선어가 구로점', '전라상회', '대찬횟집', '탐라도회'])

In [46]:
review_dic

{}

In [54]:
range(10:20)

SyntaxError: invalid syntax (3598554095.py, line 1)

In [ ]:
# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
#options = webdriver.ChromeOptions()
#options.add_argument('headless')
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)

In [23]:
url = 'https://map.naver.com/v5/search'
driver.get(url)

In [24]:
find_search_input('서울 횟집')
# searchIframe 변경
switch_frame('searchIframe')
driver.implicitly_wait(3)
page_down(40)
driver.implicitly_wait(3)
page_down(5)


WebDriverException: Message: chrome not reachable
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B978B3+2193587]
	Ordinal0 [0x00B30681+1771137]
	Ordinal0 [0x00A44070+802928]
	Ordinal0 [0x00A39AB2+760498]
	Ordinal0 [0x00A2CEAF+708271]
	Ordinal0 [0x00A31142+725314]
	Ordinal0 [0x00A355C9+742857]
	Ordinal0 [0x00A45500+808192]
	Ordinal0 [0x00A9D2C2+1168066]
	Ordinal0 [0x00A8D5C6+1103302]
	Ordinal0 [0x00A677E0+948192]
	Ordinal0 [0x00A686E6+952038]
	GetHandleVerifier [0x00E40CB2+2738370]
	GetHandleVerifier [0x00E321B8+2678216]
	GetHandleVerifier [0x00C217AA+512954]
	GetHandleVerifier [0x00C20856+509030]
	Ordinal0 [0x00B3743B+1799227]
	Ordinal0 [0x00B3BB68+1817448]
	Ordinal0 [0x00B3BC55+1817685]
	Ordinal0 [0x00B45230+1856048]
	BaseThreadInitThunk [0x7740FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77597A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77597A6E+238]


In [ ]:
store_elements = driver.find_elements(By.CSS_SELECTOR,'div._3ZU00._1rBq3 > a:nth-child(1)')

In [ ]:
len(store_elements)

In [ ]:
store_elements[26].click()

In [ ]:
switch_frame('entryIframe')

In [ ]:
switch_frame('searchIframe')

In [ ]:
driver.refresh()

In [ ]:
switch_frame('searchIframe')

In [ ]:
page_down(40)
driver.implicitly_wait(3)
page_down(5)

In [ ]:
store_elements = driver.find_elements(By.CSS_SELECTOR,'div._3ZU00._1rBq3 > a:nth-child(1)')

In [ ]:
len(store_elements)

In [ ]:
store_elements[27].click()

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
switch_frame('searchIframe')

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [44]:
page_down(20)

In [ ]:
dri